In [1]:
import numpy as np
from tqdm import tqdm
from Levenshtein import distance as levenshtein_distance, editops
import sys
from catboost import CatBoostClassifier

In [2]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import urllib

In [3]:
def multi_request(path, repeats=5, headers=None):
    for i in range(repeats):
        if headers is not None:
            req = requests.get(path, headers=headers)
        else:
            req = requests.get(path)
        if req.status_code // 100 == 2:
            break
    if req.status_code // 100 != 2:
        return req
    return req

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(ans):
    soup = BeautifulSoup(ans.text, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def parse_wikipedia(langmodel, iterations=800):
    for i in tqdm(range(iterations)):
        test = 'сапог'
        ans = multi_request(f"https://www.google.ru/complete/search?q={test}&client=gws-wiz&xssi=t&hl=ru&authuser=0&psi=lJW0YY-XBNuVjgbax7lw.1639224726752&newwindow=1&dpr=1")
        print(ans.text)
        break
        bs = BeautifulSoup(ans.text, 'html.parser')
        ans2 = multi_request('https://ru.wikipedia.org' + bs.find('li', id='n-randompage').find('a')['href'])
        txt = text_from_html(ans2)
        russian_words = ' '.join(set(re.findall(r'[а-яА-Я]+|[.]', txt)))
        langmodel.fit(russian_words)

In [4]:
with open('queries_all.txt', 'r', encoding='utf-8') as rd:
    queries = rd.readlines()

In [5]:
incorrect_queries = []
corrected_queries = []
correct_queries = []

for query in tqdm(queries):
    splitted = query.strip().split('\t')
    if len(splitted) == 2:
        corrected_queries.append(splitted[-1].lower())
        incorrect_queries.append(splitted[0].lower())
    correct_queries.append(splitted[-1].lower())

100%|██████████| 2000000/2000000 [00:03<00:00, 596515.64it/s]


In [6]:
class Language_Model:
    def __init__(self):
        self.p = {}
        self.pair_p = {}
        self.total_w = 0
        self.total_pair = 0
        
    def fit(self, cq):
        for query in cq:
            words = query.split()
            l = len(words) - 1
            self.total_w += l + 1
            for i, word in enumerate(words):
                if word in self.p:
                    self.p[word] += 1
                else:
                    self.p[word] = 1
                if i != l:
                    self.total_pair += 1
                    if words[i + 1] in self.pair_p:
                        self.pair_p[words[i + 1]][1] += 1
                        if word in self.pair_p[words[i + 1]][0]:
                            self.pair_p[words[i + 1]][0][word] += 1
                        else:
                            self.pair_p[words[i + 1]][0][word] = 1
                    else:
                        self.pair_p[words[i + 1]] = [{}, 1]
                        self.pair_p[words[i + 1]][0][word] = 1
                        
    def get_value(self, w1, w2, is_pair):
        if is_pair:
            if w2 in self.pair_p:
                if w1 in self.pair_p[w2][0]:
                    return self.pair_p[w2][0][w1] / self.pair_p[w2][1]
                else:
                    return 1 / self.total_pair
            else:
                return 1 / self.total_pair
        else:
            if w1 in self.p:
                return self.p[w1] / self.total_w
            else:
                return 1 / self.total_w
    
    def predict(self, query):
        words = query.strip().split()
        l = len(words) - 1
        probability = 1
        for i, word in enumerate(words):
            if i != l:
                probability *= self.get_value(word, words[i + 1], True)
            else:
                probability *= self.get_value(word, None, False)
        return probability

In [7]:
class Error_Model:
    def __init__(self):
        self.bigrams = {}
        self.unigrams = {}
        self.unitransforms = 0
        self.bitransforms = 0
    
    def fill_unigram(self, key1, key2):
        if key1 in self.unigrams:
            if key2 in self.unigrams[key1][0]:
                self.unigrams[key1][0][key2] += 1
            else:
                self.unigrams[key1][0][key2] = 1
            self.unigrams[key1][1] += 1
        else:
            self.unigrams[key1] = [{}, 1]
            self.unigrams[key1][0][key2] = 1
    
    def fill_bigram(self, key1, key2):
        if key1 in self.bigrams:
            if key2 in self.bigrams[key1][0]:
                self.bigrams[key1][0][key2] += 1
            else:
                self.bigrams[key1][0][key2] = 1
            self.bigrams[key1][1] += 1
        else:
            self.bigrams[key1] = [{}, 1]
            self.bigrams[key1][0][key2] = 1
    
    def fit(self, cqueries, wqueries):
        i = 0
        for cq, wq in zip(cqueries, wqueries):
            editions = editops(wq, cq)
            l = len(wq) - 1
            for i, ed in enumerate(editions):
                self.unitransforms += 1
                if ed[0][0] == 'i':
                    self.fill_unigram('', cq[ed[2]])
                    self.bitransforms += 1
                    if ed[1] != 0:
                        self.fill_bigram(wq[ed[1] - 1], wq[ed[1] - 1] + cq[ed[2]])
                    if l != ed[1] - 1:
                        self.fill_bigram(wq[ed[1]], cq[ed[2]] + wq[ed[1]])
                elif ed[0][0] == 'd':
                    self.fill_unigram(wq[ed[1]], '')
                    if ed[1] != 0:
                        self.bitransforms += 1
                        self.fill_bigram(wq[ed[1]-1 : ed[1]+1], wq[ed[1]-1])
                    if ed[1] != l:
                        self.bitransforms += 1
                        self.fill_bigram(wq[ed[1] : ed[1]+2], wq[ed[1]+1])
                else:
                    self.fill_unigram(wq[ed[1]], cq[ed[2]])
                    if ed[1] != 0:
                        self.bitransforms += 1
                        self.fill_bigram(wq[ed[1]-1 : ed[1]+1], wq[ed[1]-1] + cq[ed[2]])
                    if ed[1] != l:
                        self.bitransforms += 1
                        self.fill_bigram(wq[ed[1] : ed[1]+2], cq[ed[2]] + wq[ed[1]+1])
    
    def try_unigram(self, key1, key2):
        if key1 in self.unigrams:
            if key2 in self.unigrams[key1][0]:
                return self.unigrams[key1][0][key2] / self.unigrams[key1][1]
            else:
                return 1 / self.unigrams[key1][1]
        return 1 / self.unitransforms
    
    def try_bigram(self, key1, key2):
        if key1 in self.bigrams:
            if key2 in self.bigrams[key1][0]:
                return self.bigrams[key1][0][key2] / self.bigrams[key1][1]
            else:
                return 1 / self.bigrams[key1][1]
        return 1 / self.bitransforms
    
    def predict(self, orig, fix):
        orig = orig.lower()
        fix = fix.lower()
        editions = editops(orig, fix)
        l = len(orig) - 1
        probability = 1 # 1/5, 2/5, 2/5
        for i, ed in enumerate(editions):
            probs = []
            if ed[0][0] == 'i':
                probs.append(self.try_unigram('', fix[ed[2]]))
                if ed[1] != 0:
                    probs.append(self.try_bigram(orig[ed[1] - 1], orig[ed[1] - 1] + fix[ed[2]]))
                if l != ed[1] - 1:
                    probs.append(self.try_bigram(orig[ed[1]], fix[ed[2]] + orig[ed[1]]))
            elif ed[0][0] == 'd':
                probs.append(self.try_unigram(orig[ed[1]], ''))
                if ed[1] != 0:
                    probs.append(self.try_bigram(orig[ed[1]-1:ed[1]+1], orig[ed[1]-1]))
                elif ed[1] != l:
                    probs.append(self.try_bigram(orig[ed[1] : ed[1]+2], orig[ed[1]+1]))
            else:
                probs.append(self.try_unigram(orig[ed[1]], fix[ed[2]]))
                if ed[1] != 0:
                    probs.append(self.try_bigram(orig[ed[1]-1 : ed[1]+1], orig[ed[1]-1] + fix[ed[2]]))
                if ed[1] != l:
                    probs.append(self.try_bigram(orig[ed[1] : ed[1]+2], fix[ed[2]] + orig[ed[1]+1]))
            if len(probs) == 1:
                probability *= probs[0]
            elif len(probs) == 2:
                probability *= (probs[0] * 1 / 3 + probs[1] * 2 / 3)
            else:
                probability *= (probs[0] * 1 / 5 + probs[1] * 2 / 5 + probs[2] * 2 / 5)
        return probability

In [8]:
Err_model = Error_Model()
LangModel = Language_Model()
GooLangModel = Language_Model()

In [9]:
LangModel.fit(correct_queries)

In [10]:
Err_model.fit(corrected_queries, incorrect_queries)

In [11]:
def get_weighted_index(n):
    tmp = np.arange(n)
    return np.random.choice(tmp, size=None, p=(n - tmp) / (tmp.sum() + n))
def gbs(cresult, orig, n_features=30):
    l = []
    for i in range(n_features):
        s = ""
        for elem in cresult:
            s += elem[get_weighted_index(len(elem))][0]
        l.append(Err_model.predict(orig, s))
    return np.asarray(l)

def get_best_fix(fixes):
    s = ""
    for word in fixes:
        s += word[0][0] + ' '
    return s
class CorrectWordGenerator:
    
    def __init__(self, errmodel, langmodel, thresh=1e-2):
        self.pwords = []
        self.em = errmodel
        self.lm = langmodel
        self.thresh = thresh
        self.best_p = thresh
    
    def get_value_fix(self, p, fix, word):
        lv = self.lm.get_value(fix, None, False)
        return np.log2(p) + np.log2(lv) * (1 + abs(len(fix) - len(word)) / len(word)) ** 2
    
    def recursive_word_fix(self, word, i, cur_p, fix):
        if i == len(word):
            val = self.get_value_fix(cur_p, fix, word)
            if val > self.best_p:
                self.best_p = val
            yield [fix, cur_p]
            return
        if self.em.predict(word[:len(fix)], fix) < self.thresh:
            return
        if word[i] in self.em.unigrams:
            for key in self.em.unigrams[word[i]][0]:
                tcur_p = cur_p * self.em.unigrams[word[i]][0][key] / self.em.unigrams[word[i]][1]
                if tcur_p > self.thresh:
                    yield from self.recursive_word_fix(word, i + 1, tcur_p, fix + key)
        if word[i] in self.em.bigrams:
            for key in self.em.bigrams[word[i]][0]:
                tcur_p = cur_p * self.em.bigrams[word[i]][0][key] / self.em.bigrams[word[i]][1]
                if tcur_p > self.thresh:
                    yield from self.recursive_word_fix(word, i + 1, tcur_p, fix + key)               
        if i != 0:
            tmp = word[i-1:i+1]
            if tmp in self.em.bigrams:
                for key in self.em.bigrams[tmp][0]:
                    tcur_p = cur_p * self.em.bigrams[tmp][0][key] / self.em.bigrams[tmp][1]
                    if tcur_p > self.thresh:
                        yield from self.recursive_word_fix(word, i + 1, tcur_p, fix + key)
        if i != len(word) - 1:
            tmp = word[i:i+2]
            if tmp in self.em.bigrams:
                for key in self.em.bigrams[tmp][0]:
                    tcur_p = cur_p * self.em.bigrams[tmp][0][key] / self.em.bigrams[tmp][1]
                    if tcur_p > self.thresh:
                        yield from self.recursive_word_fix(word, i + 1, tcur_p, fix + key)
        yield from self.recursive_word_fix(word, i + 1, cur_p, fix + word[i])
    
    def clean_duplicates(self, pcorr):
        clean_pcorr = []
        for i in range(len(pcorr)):
                if i == 0:
                    last_word = pcorr[i][0]
                    clean_pcorr.append(pcorr[i])
                else:
                    if pcorr[i][0] != last_word:
                        clean_pcorr.append(pcorr[i])
                        last_word = pcorr[i][0]    
        return clean_pcorr
    
    def get_possible_fixes(self, query):
        words = query.lower().strip().split()
        pcorrections = []
        for i, word in enumerate(words):
            pcorrections.append(list(self.recursive_word_fix(word, 0, 1, '')))
            for j in range(len(pcorrections[i])):
                pcorrections[i][j].append(self.lm.get_value(pcorrections[i][j][0], None, False))
                pcorrections[i][j].append(np.log2(pcorrections[i][j][1]) + np.log2(pcorrections[i][j][2]) * (1 + abs(len(pcorrections[i][j][0]) - len(word)) / len(word)) ** 2)
        for i in range(len(pcorrections)):
            pcorrections[i] = sorted(self.clean_duplicates(sorted(sorted(pcorrections[i], key= lambda x: x[-1], reverse=True), key= lambda x: x[0])), key= lambda x: x[-1], reverse=True)
        return pcorrections
            

In [12]:
CorrModel = CorrectWordGenerator(Err_model, LangModel)

In [13]:
def get_letter(mistakes):
    keys = list(mistakes.keys())
    vals = np.asarray(list(mistakes.values()))
    return np.random.choice(keys, 1, p=vals / vals.sum())[0]
def gen_incorrect_queries(cq):
    wrong_queries = []
    c_queries = []
    for i, q in enumerate(tqdm(cq)):
        a = []
        for j in range(np.random.randint(1, 3)):
            a.append(np.random.randint(len(q)))
            while q[a[-1]] == ' ':
                a[-1] = np.random.randint(len(q))
        s = ""
        for j in range(len(q)):
            if j in a:
                if q[j] in Err_model.unigrams:
                    s += get_letter(Err_model.unigrams[q[j]][0])
                else:
                    tl = list(Err_model.unigrams.keys())
                    s += tl[np.random.randint(len(tl))]
            else:
                s += q[j]
        if s != q:
            wrong_queries.append(s)
            c_queries.append(q)
    return wrong_queries, c_queries

In [14]:
pair_queries = gen_incorrect_queries(correct_queries)

100%|██████████| 2000000/2000000 [04:43<00:00, 7061.01it/s]


In [15]:
class FixNonePredictor:
    
    def __init__(self, it=200):
        self.model = CatBoostClassifier(iterations=it)
        
    def get_features(self, orig, fix):
        return np.array([len(orig.split()), LangModel.predict(orig), LangModel.predict(fix), Err_model.predict(orig, fix)])
    
    def fit(self, wrong_queries, correct_queries):
        self.X_train = np.zeros((len(wrong_queries) * 2, 4))
        self.y_train = np.zeros((len(wrong_queries) * 2, ))
        i = 0
        for wquery, cquery in zip(tqdm(wrong_queries), correct_queries):
            self.X_train[2 * i] = self.get_features(wquery, cquery)
            self.X_train[2 * i + 1] = self.get_features(cquery, wquery)
            self.y_train[2 * i] = 1
            self.y_train[2 * i + 1] = 0
            i += 1
        self.model.fit(self.X_train, self.y_train)
    
    def predict(self, orig_queries):
        self.orig_queries = orig_queries
        self.fix_queries = []
        for query in orig_queries:
            self.fix_queries.append(get_best_fix(CorrModel.get_possible_fixes(query.strip())).strip())
        X_test = np.zeros((len(orig_queries), 4))
        i = 0
        for orig, fix in zip(orig_queries, self.fix_queries):
            X_test[i] = self.get_features(orig, fix)
            i += 1
        return self.model.predict(X_test)

In [16]:
fixclf = FixNonePredictor(200)

In [17]:
fixclf.fit(pair_queries[0], pair_queries[1])

100%|██████████| 2000000/2000000 [01:22<00:00, 24332.08it/s]


Learning rate set to 0.5
0:	learn: 0.1546446	total: 535ms	remaining: 1m 46s
1:	learn: 0.0921814	total: 979ms	remaining: 1m 36s
2:	learn: 0.0737151	total: 1.44s	remaining: 1m 34s
3:	learn: 0.0587625	total: 1.89s	remaining: 1m 32s
4:	learn: 0.0488183	total: 2.35s	remaining: 1m 31s
5:	learn: 0.0458015	total: 2.82s	remaining: 1m 31s
6:	learn: 0.0439416	total: 3.26s	remaining: 1m 29s
7:	learn: 0.0394935	total: 3.72s	remaining: 1m 29s
8:	learn: 0.0369250	total: 4.17s	remaining: 1m 28s
9:	learn: 0.0362689	total: 4.64s	remaining: 1m 28s
10:	learn: 0.0359325	total: 5.03s	remaining: 1m 26s
11:	learn: 0.0347341	total: 5.46s	remaining: 1m 25s
12:	learn: 0.0343481	total: 5.85s	remaining: 1m 24s
13:	learn: 0.0339178	total: 6.27s	remaining: 1m 23s
14:	learn: 0.0337189	total: 6.7s	remaining: 1m 22s
15:	learn: 0.0333874	total: 7.17s	remaining: 1m 22s
16:	learn: 0.0332204	total: 7.55s	remaining: 1m 21s
17:	learn: 0.0330773	total: 8.01s	remaining: 1m 20s
18:	learn: 0.0330278	total: 8.4s	remaining: 1m 20s

In [558]:
with open('model.bin', 'wb') as f:
    pickle.dump(fixclf, f)
with open('lang.bin', 'wb') as f:
    pickle.dump(LangModel, f)
with open('error.bin', 'wb') as f:
    pickle.dump(Err_model, f)

In [560]:
import pickle
with open('model.bin', 'rb') as f:
    flxclf = pickle.load(f)
with open('error.bin', 'rb') as f:
    Err_model = pickle.load(f)
with open('lang.bin', 'rb') as f:
    LangModel = pickle.load(f)

for q in sys.stdin:
    q1 = q.lower().strip()
    if not fixclf.predict([q1])[0]:
        print(q)
    else:
        print(fixclf.fix_queries)
    

100%|██████████| 1/1 [00:00<00:00, 7256.58it/s]

HU^Y HUUUU


In [24]:
fixclf.predict(['Взлом игры на рыбалку вк'])
fixclf.fix_queries

['взлом игры на рыбалку вк']

In [ ]:
total = 0
for i, query in enumerate(queries):
    splitted = query.strip().split('\t')
    if i == 100:
        break
    q1 = splitted[0].lower().strip()
    if not fixclf.predict([q1])[0]:
        total += int(splitted[0] == splitted[-1])
    else:
        total += int(fixclf.fix_queries[0] == splitted[-1])
    print(total / (i + 1), splitted[0], '|', splitted[-1])
total / 100

In [22]:
CorrModel.get_possible_fixes("")

[[['папа', 0.012939576834810528, 0.0001124083420371737, -19.391029027245658],
  ['по', 0.03224285507036124, 0.004119899555117223, -22.782020711904977],
  ['фшпо', 1, 1.3381945480615918e-07, -22.83320879253681],
  ['фшпа', 0.5674223702589449, 1.3381945480615918e-07, -23.65071385837437],
  ['фщпо', 0.47013274336283184, 1.3381945480615918e-07, -23.922068723926934],
  ['вшпо', 0.3475409836065574, 1.3381945480615918e-07, -24.35795377042386],
  ['фшшо', 0.3333333333333333, 1.3381945480615918e-07, -24.418171293257966],
  ['фщпа', 0.2667638355752783, 1.3381945480615918e-07, -24.739573789764492],
  ['вшпа', 0.19720252868015792, 1.3381945480615918e-07, -25.17545883626142],
  ['фшша', 0.18914079008631496, 1.3381945480615918e-07, -25.235676359095525],
  ['фппо', 0.16666666666666666, 1.3381945480615918e-07, -25.418171293257963],
  ['вщпо', 0.1633903960539678, 1.3381945480615918e-07, -25.44681370181398],
  ['шпон', 0.011546655155107728, 1.8734723672862285e-06, -25.46223506921968],
  ['фщшо', 0.15671